In [ ]:
import numpy as np
import rebound
from scipy.integrate import odeint
import matplotlib.pyplot as plt
from celmech.disturbing_function import df_coefficient_Ctilde
from celmech.disturbing_function import evaluate_df_coefficient_dict

In [ ]:
def wrapRad(theta, k):
    return (theta + k*np.pi) % (2*k*np.pi) - k*np.pi

$$ param: \qquad
r = \frac{j_2}{j_1},\qquad
\epsilon_1 = \mu_1 |C_1| e^{k_1},\qquad
\epsilon_2 = \mu_2 |C_2| e^{k_2}, \qquad
\nu = \frac{1}{\sqrt{3}}\frac{n_{r2}-n_{r1}}{n_{r1}} = \frac{1}{\sqrt{3}}\left(\frac{j_1(j_2+k_2)}{j_2(j_1-k_1)}\frac{n_2}{n_1}-1\right)$$

In [ ]:
def param2OE(param, userDefined):
    r, nu, epsilon1, epsilon2 = param
    ecc, P1, j1, k1, k2 = userDefined
    
    j2 = r*j1
    
    #calculate C1 and C2
    coeff1 = (j1,-(j1-k1),0,-k1,0,0)
    C1 = df_coefficient_Ctilde(*coeff1,*(0,0,0,0))
    alpha1 = ((j1-k1)/j1)**(2/3)
    Cval1 = evaluate_df_coefficient_dict(C1,alpha1)

    coeff2 = (j2+k2,-j2,-k2,0,0,0)
    C2 = df_coefficient_Ctilde(*coeff2,*(0,0,0,0))
    alpha2 = (j2/(j2+k2))**(2/3)
    Cval2 = evaluate_df_coefficient_dict(C2,alpha2)

    # get mu1 and mu2
    mu1 = epsilon1 / (abs(Cval1) * ecc**k1)
    mu2 = epsilon2 / (abs(Cval2) * ecc**k2)
    
    # get P2
    P2 = P1 * j1*(j2+k2) / (j2*(j1-k1)*(np.sqrt(3)*nu + 1))
    print(Cval1, Cval2)

    return mu1, mu2, P1, P2

$$ ICs: \qquad
\phi = j_1(\lambda-n_{r1}t),\qquad 
p = -\sqrt{3}\left[\left(\frac{n_{r1}}{2\pi/P_0}\right)^{1/3}-1\right]$$

In [ ]:
def param2OE_IC(j1, k1, P1, IC):
    n_r1 = (j1-k1)*2*np.pi / (j1*P1)
    return [[phi_0/j1 ,(1-p_0/np.sqrt(3))**3 * (2*np.pi/n_r1)] for phi_0, p_0 in IC]

def OE2param_IC(j1, k1, P1, IC):
    n_r1 = (j1-k1)*2*np.pi / (j1*P1)
    return [[j1*lambda_0, -np.sqrt(3) * ((n_r1 / (2*np.pi/P_0))**(1/3) - 1)] for lambda_0, P_0 in IC]

In [ ]:
# ecc, P1, j1, k1, j2, k2 = 0.05, 1, 5, 2, 2, 1
# r, epsilon1, epsilon2 = j2/j1, 9.5e-7, 3.3e-6
# nu = 10*np.sqrt(epsilon1)

s = 1/4000
nu = 0.0015
ecc, P1, j1, k1, k2 = 0.05, 1, 5, 1, 1
r, epsilon1, epsilon2 = 1, s**2, 0.4*s**2
j2 = r*j1
mu1, mu2, P1, P2 = param2OE((r, nu, epsilon1, epsilon2), (ecc, P1, j1, k1, k2))

In [ ]:
# set lambda_0 and P_0
ic_param = [[0,x] for x in np.linspace(-nu, 1.5*nu, 20)]
ic_OE = param2OE_IC(j1, k1, P1, ic_param)

In [ ]:
span=1

In [ ]:
# Plot surface of section by rebound and by Hamiltonian

#---------------------------------REBOUND-------------------------------------
n_r1 = (j1-k1) * 2*np.pi / (j1*P1)
n_r2 = (j2+k2) * 2*np.pi / (j2*P2)
omega1, omega2, omega0 = 0, 0, 0
    
for lambda_0, P_0 in ic_OE:
    # start simulation
    sim = rebound.Simulation()
    sim.units = ('yr', 'AU', 'Msun')

    # add star, planet 1, planet 2
    sim.add(m=1.)
    sim.add(m=mu1, P=P1, omega=omega1, l=np.pi/(j1-k1))
    sim.add(m=mu2, P=P2, omega=omega2, l=0)
    ps = sim.particles
    ps[1].r = ps[1].a*(ps[1].m/3/ps[0].m)**(1/3)
    ps[2].r = ps[2].a*(ps[2].m/3/ps[0].m)**(1/3)
    
    # add test particle
    sim.add(P=P_0, l=lambda_0, e=ecc, omega=omega0)
    sim.move_to_com()
    sim.integrator = "whfast"
    sim.dt = sim.particles[1].P/20
    sim.collision = "direct"
    
    # track how system develops in time
    num = 201
    time = np.linspace(0,num-1,num)* 2*np.pi/(r*nu) / (n_r1*j1*np.sqrt(3))  # Note: t is the unscaled time

    phi, p = np.zeros(num), np.zeros(num)
    for i,t in enumerate(time):
        try:
            sim.integrate(t)
        except rebound.Collision:
            print('Collision')
        info = sim.particles[3].calculate_orbit(primary=sim.particles[0])
        
        phi[i] = wrapRad(j1*(info.l - n_r1*t) - info.omega, span)
        p[i] = -np.sqrt(3)*((n_r1 / info.n)**(1/3) - 1)
    plt.scatter(phi,p,[1],color='black')
plt.show()



In [ ]:
def hamilton(y, t, epsilon1, epsilon2, r, nu):
    phi, p = y
    dydt = [p, -epsilon1*np.sin(phi) - r*epsilon2*np.sin(r*(phi-nu*t))]
    return dydt

In [ ]:
t = np.linspace(0,num-1,num)*2*np.pi/(r*nu)  # Note: t is the rescaled time
for y0 in ic_param:
    sol = odeint(hamilton, y0, t, args=(epsilon1,epsilon2,r,nu))
    phi = (sol[:,0] + span*np.pi*np.ones(num)) % (2*span*np.pi) - span*np.pi*np.ones(num)  # wrap into [-pi,pi]
    plt.scatter(phi, sol[:,1], [1], color='black')
plt.show()